In [1]:
from cloudpathlib import AnyPath
from simple_logger import get_logger
logger = get_logger(__name__)
from app.settings import settings

#root_path = AnyPath("/home/john/projects/labcats/new_process_data")
root_path = AnyPath(settings.root_path)

data_root = root_path / "data"

import app.file_utils as fu
from importlib import reload
reload(fu)

fu.maybe_mkdir(data_root)

import app.google_sheets as gs

target_locations_df = gs.load_dataframe_from_google_sheet (
    file_id=settings.target_locations_sheet_id
)

target_locations = target_locations_df.Locations.tolist()
logger.info(f"Target locations: {target_locations}")

[2026-02-18T10:38:59] INFO app.file_utils: Bucket already exists: categorum-test4


/home/john/.pyenv/versions/3.12.6/envs/hyperlocal/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2026-02-18T10:39:01] INFO __main__: Target locations: ['Peckham', 'Gorton and Denton', 'Kensington and Bayswater', 'Bolsover', 'Makerfield']


In [4]:
from app.sync_downloads import sync_all_downloads
from datetime import datetime, timezone

sync_all_downloads(
    data_root,
    ['Gorton and Denton',],
    first_start=datetime(2026, 2, 2, tzinfo=timezone.utc),
)

[2026-02-18T14:19:57] INFO app.sync_downloads: Created ES client
[2026-02-18T14:19:57] INFO app.sync_downloads: Checking week | extra={'start': '2026-02-02', 'end': '2026-02-09'}
[2026-02-18T14:20:00] INFO app.process_week: Downloaded posts for period | extra={'index': 'dalmation-fb-posts', 'hits': 2054}
[2026-02-18T14:20:08] INFO app.process_week: Downloaded posts for period | extra={'index': 'dalmation-fb-comments', 'hits': 5571}
[2026-02-18T14:20:09] INFO app.sync_downloads: Downloaded entries | extra={'posts': 2054, 'comments': 5571}
[2026-02-18T14:20:09] INFO app.posts_and_comments: Merged posts and top comments; resulting rows: 389
[2026-02-18T14:20:09] INFO app.sync_downloads: Checking week | extra={'start': '2026-02-09', 'end': '2026-02-16'}
[2026-02-18T14:20:11] INFO app.process_week: Downloaded posts for period | extra={'index': 'dalmation-fb-posts', 'hits': 3360}
[2026-02-18T14:20:12] INFO app.process_week: Downloaded posts for period | extra={'index': 'dalmation-fb-comments

In [4]:
import re
import app.post_processing as pp
import app.file_utils as fu
from importlib import reload
reload(pp)

#locations_to_process = ['Gorton and Denton',]
locations_to_process = None

# Discover posts files that don't have a corresponding processed file yet
if locations_to_process:
    # Slugify location names to match the filenames on disk/GCS
    loc_slugs = [fu.file_name_to_slug(loc) for loc in locations_to_process]
    loc_group = "|".join(re.escape(s) for s in loc_slugs)
    posts_pattern = re.compile(rf"^posts_({loc_group})_(\d{{4}}-\d{{2}}-\d{{2}})_to_(\d{{4}}-\d{{2}}-\d{{2}})\.feather$")
else:
    posts_pattern = re.compile(r"^posts_(.+)_(\d{4}-\d{2}-\d{2})_to_(\d{4}-\d{2}-\d{2})\.feather$")

unprocessed = []
for f in data_root.iterdir():
    m = posts_pattern.match(f.name)
    if not m:
        continue
    location_slug, date_from, date_to = m.groups()
    processed_name = f"processed_{location_slug}_{date_from}_to_{date_to}.feather"
    if not (data_root / processed_name).exists():
        unprocessed.append((date_from, date_to, location_slug, f))

# Sort chronologically (earliest first)
unprocessed.sort()

print(f"Found {len(unprocessed)} unprocessed posts file(s)")
for date_from, date_to, location_slug, input_path in unprocessed:
    print(f"  {input_path.name}")

for date_from, date_to, location_slug, input_path in unprocessed:
    print(f"\nProcessing {input_path.name} ...")
    pipeline = pp.PostProcessingPipeline(
        categories_path=root_path/"categories_to_study.json",
        tags_path=data_root/"tags"/location_slug,
        intermediary_path=AnyPath("/home/john/projects/hyperlocal/junk"),
    )
    processed_df = pipeline.process(input_path, save_intermediary_files=False)
    output_filename = f"processed_{location_slug}_{date_from}_to_{date_to}.feather"
    fu.write_feather_to_anypath(processed_df, data_root / output_filename)
    print(f"  Saved {output_filename}")


Found 4 unprocessed posts file(s)
  posts_gorton-and-denton_2026-02-09_to_2026-02-16.feather
  posts_kensington-and-bayswater_2026-02-09_to_2026-02-16.feather
  posts_makerfield_2026-02-09_to_2026-02-16.feather
  posts_peckham_2026-02-09_to_2026-02-16.feather

Processing posts_gorton-and-denton_2026-02-09_to_2026-02-16.feather ...
[2026-02-17T23:06:48] INFO app.post_processing: Stage 1: starting
[2026-02-17T23:06:48] INFO app.post_processing: Stage 1: using model | extra={'model': 'google/gemini-2.5-flash'}
[2026-02-17T23:06:48] INFO app.ai_wrapper: Starting iteration 1
[2026-02-17T23:06:48] INFO app.ai_wrapper: Processing 363 rows in this iteration
[2026-02-17T23:06:54] INFO app.ai_wrapper: Iteration 1 complete: Successfully processed rows: 363 | Remaining invalid rows: 0
[2026-02-17T23:06:54] INFO app.ai_wrapper: Starting iteration 2
[2026-02-17T23:06:54] INFO app.ai_wrapper: All rows are either valid or have reached max iterations
[2026-02-17T23:06:54] INFO app.ai_wrapper: Finished 

In [ ]:
from importlib import reload
import app.process_week as pw
from datetime import timezone, timedelta
import app.tag_manager as tm
import app.file_utils as fu
start_dt, end_dt = pw.get_most_recent_week(tz=timezone.utc, week_start=0, weeks_ago=0)
import app.google_sheets as gs
reload (gs)

import app.settings as settings
reload(settings)
from app.settings import settings

output_drive_root = settings.output_drive_root

field_map = {
    "timestamp": "Date posted",
    "url": "Link",
    "body": "Post text",
    "category": "Category",
    "tag": "Tag",
    "tag_description": "Tag description",
    "comments": "# comments"
}

for location in target_locations:
    print (location)
    fin = fu.file_name_to_slug(data_root / f"processed_{location}_{start_dt.strftime('%Y-%m-%d')}_to_{end_dt.strftime('%Y-%m-%d')}.feather")
    df = fu.read_feather_from_anypath(fin)
    df = tm.merge_tags(df,data_root / "tags" / location)
    df_out = df[field_map.keys()].rename(columns=field_map)
    out_folder_id = gs.get_drive_folder_by_name(f'{location}/data_sheets', parent_drive_folder_id=output_drive_root, create_if_missing=True)
    out_file_id = gs.upload_dataframe_to_google_sheet(
        df_out, 
        out_folder_id, 
        spreadsheet_title=f'output week starting {start_dt.strftime("%Y-%m-%d")}',
        max_column_width=500,
        max_row_height=200
    )

Peckham
[2026-02-18T10:09:02] INFO app.tag_manager: TagManager initialized | extra={'path': 'gs://categorum-test3/hyperlocal/data/tags/peckham/tag_record.csv'}
[2026-02-18T10:09:02] DEBUG app.tag_manager: Saving tag record | extra={'path': 'gs://categorum-test3/hyperlocal/data/tags/peckham/tag_record.csv', 'rows': 12}
Gorton and Denton
[2026-02-18T10:09:14] INFO app.tag_manager: TagManager initialized | extra={'path': 'gs://categorum-test3/hyperlocal/data/tags/gorton-and-denton/tag_record.csv'}
[2026-02-18T10:09:14] DEBUG app.tag_manager: Saving tag record | extra={'path': 'gs://categorum-test3/hyperlocal/data/tags/gorton-and-denton/tag_record.csv', 'rows': 155}
Kensington and Bayswater
[2026-02-18T10:09:26] INFO app.tag_manager: TagManager initialized | extra={'path': 'gs://categorum-test3/hyperlocal/data/tags/kensington-and-bayswater/tag_record.csv'}
[2026-02-18T10:09:26] DEBUG app.tag_manager: Saving tag record | extra={'path': 'gs://categorum-test3/hyperlocal/data/tags/kensington-a